In [23]:
import json
import logging
import views

logging.basicConfig(
    level=logging.DEBUG,
    #level=logging.INFO, # uncomment this and comment debug above for less yelling in red
    format=views.config.LOGFMT,
)

In [24]:
import pandas as pd

In [25]:
# DATASETS is a dictionary of Dataset objects.
from views import DATASETS
# These are the building blocks of the modelling interface
from views import Ensemble, Model, Downsampling, Period
# These are model specifications from the specfiles
from views.specs.models import cm as model_specs_cm, pgm as model_specs_pgm
from views.specs.periods import get_periods, get_periods_by_name
# Utils
from views.utils import db, io, data as datautils
from views.utils.data import assign_into_df

In [26]:
# These are defined in 
from views.apps.pipeline.models_cm import all_cm_models_by_name
from views.apps.pipeline.models_pgm import all_pgm_models_by_name

In [27]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [114]:
dataset = views.DATASETS["cm_africa_imp_0"]
df = dataset.df
df2=df

[2020-12-23 13:59:50,246] - views.utils.io:65 - DEBUG - Reading parquet at /home/kyle/code/Views2/OpenViEWS2/storage/data/datasets/cm_africa_imp_0.parquet with cols None
[2020-12-23 13:59:50,876] - views.utils.io:72 - DEBUG - Finished reading parquet from /home/kyle/code/Views2/OpenViEWS2/storage/data/datasets/cm_africa_imp_0.parquet.


In [29]:
type(df)

pandas.core.frame.DataFrame

In [30]:
df['wdi_sp_pop_totl'].describe()


count    3.304800e+04
mean     1.606108e+07
std      2.433926e+07
min      6.326100e+04
25%      2.119275e+06
50%      8.450630e+06
75%      1.987708e+07
max      1.958747e+08
Name: wdi_sp_pop_totl, dtype: float64

In [9]:
df.tail()


acled_count_ns  acled_count_os  acled_count_pr  \
month_id country_id                                                   
612      242                    0.0             0.0             0.0   
         243                    0.0             0.0             0.0   
         244                    0.0             0.0             0.0   
         245                    0.0             0.0             0.0   
         246                    0.0             0.0             0.0   

                     acled_count_sb  acled_dummy_ns  acled_dummy_os  \
month_id country_id                                                   
612      242                    0.0               0               0   
         243                    0.0               0               0   
         244                    0.0               0               0   
         245                    0.0               0               0   
         246                    0.0               0               0   

                     acled_dummy_pr  acled_dummy_sb  cdum_1  cdum_10  ...  \
month_id country_id                                                   ...   
612      242                      0               0       0        0  ...   
         243                      0               0       0        0  ...   
         244                      0               0       0        0  ...   
         245                      0               0       0        0  ...   
         246                      0               0       0        0  ...   

                     wdi_tx_val_trvl_zs_wt  wdi_vc_btl_deth  wdi_vc_idp_nwcv  \
month_id country_id                                                            
612      242                     61.237308        81.692308           1000.0   
         243                     43.453430       207.000000             35.0   
         244                      3.752579         8.000000          36000.0   
         245                     70.229657       246.000000          84000.0   
         246                      0.711187       536.000000         259000.0   

                     wdi_vc_idp_nwds  wdi_vc_idp_tocv  wdi_vc_ihr_psrc_fe_p5  \
month_id country_id                                                            
612      242                 11000.0           3400.0                    2.5   
         243                   200.0         433000.0                    0.6   
         244                  6600.0            700.0                    1.4   
         245                272000.0        2134000.0                    2.9   
         246                294000.0        1352000.0                    2.0   

                     wdi_vc_ihr_psrc_ma_p5  wdi_vc_ihr_psrc_p5  \
month_id country_id                                              
612      242                          11.5                 6.2   
         243                           3.7                 2.1   
         244                          10.3                 9.3   
         245                          14.0                 5.2   
         246                           9.9                13.9   

                     wdi_vc_pkp_totl_un  year  
month_id country_id                            
612      242                     2622.0  2030  
         243                      361.0  2030  
         244                      562.0  2030  
         245                    16692.0  2030  
         246                    15415.0  2030  

[5 rows x 3157 columns]

In [9]:
df['wdi_sp_pop_totl'].head(15)

month_id  country_id
1         40            2.838470e+05
          41            8.033650e+06
          42            1.105612e+07
          43            1.853001e+06
          47            6.822843e+06
          48            4.871435e+06
          49            7.816770e+05
          50            7.090126e+06
          52            5.583165e+06
          53            3.388495e+06
          54            6.372460e+05
          55            3.589600e+05
          56            1.733416e+06
          57            2.259068e+07
          67            1.922166e+07
Name: wdi_sp_pop_totl, dtype: float64

In [11]:
df['wdi_ag_prd_food_xd'].tail(15)

month_id  country_id
612       169           119.32
          170            92.56
          172           121.29
          173            91.81
          174           103.99
          213           117.43
          214           155.20
          222           124.98
          235            98.31
          237           125.56
          242           168.95
          243           118.49
          244           120.03
          245           106.86
          246            96.13
Name: wdi_ag_prd_food_xd, dtype: float64

acled_count_ns              0
acled_count_os              0
acled_count_pr              0
acled_count_sb              0
acled_dummy_ns              0
                         ... 
wdi_vc_ihr_psrc_fe_p5     2.6
wdi_vc_ihr_psrc_ma_p5    14.7
wdi_vc_ihr_psrc_p5       11.5
wdi_vc_pkp_totl_un        871
year                     2030
Name: (612, 40), Length: 3157, dtype: object

In [31]:
run_id = "d_2020_10_01_prelim"

In [38]:
periods = get_periods(run_id) # as a list
periods_by_name = get_periods_by_name(run_id)# as a dict

period_a = periods_by_name["A"]

period_b = periods_by_name["B"]
period_c = periods_by_name["C"]
period_c
period_c.predict_start
period_c.predict_end

[2020-12-23 11:20:59,780] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml
[2020-12-23 11:20:59,818] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml


527

In [95]:
# arrayOfPrediction = [*range(period_c.predict_start, period_c.predict_end, 1)]
arrayOfPrediction = [*range(330, period_c.predict_end, 1)]

print(arrayOfPrediction)

[330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526]


In [93]:
Countries=[{"Cote d'Ivoire":40},{'Ghana':41},{'Liberia':42},{'Morocco':43},{'Burkina Faso':46},{'Guinea':47},{'Guinea-Bissau':48},{'Mali':49},{'Mauritania':50},{'Senegal':51},{'Sierra Leone':52},{'The Gambia':53},{'Djibouti':54},{'Eritrea':55},{'Ethiopia':56},{'Sudan':58},{'Israel':61},{'Jordan':62},{'Algeria':67},{'Cameroon':69},{'Central African Republic':70},{'Libya':71},{'Tunisia':73},{'Benin':74},{'Chad':75},{'Equatorial Guinea':76},{'Niger':78},{'Nigeria':79},{'Sao Tome and Principe':80},{'Togo':81},{'Egypt':92},{'Oman':120},{'Somalia':121},{'Yemen':125},{'Bahrain':128},{'Kuwait':130},{'Qatar':131},{'Saudi Arabia':132},{'United Arab Emirates':133},{'Botswana':155},{'Burundi':156},{'Rwanda':158},{'Zambia':160},{'Zimbabwe':161},{'Comoros':162},{'Lesotho':163},{'Malawi':164},{'Mozambique':165},{'South Africa':166},{'Swaziland':167},{'Angola':168},{'Congo':169},{'Congo, DRC':170},{'Gabon':172},{'Namibia':173},{'Madagascar':175},{'Seychelles':177},{'Zanzibar':194},{'Ethiopia':195},{'South Africa':196},{'Egypt':197},{'Egypt (United Arab Republic)':199},{'Yemen Arab Republic':200},{"Yemen People's Republic":201},{'Mali Federation':214},{'Cameroon':215},{'Nigeria':216},{'Libya':217},{'Chad':218},{'Morocco':219},{'Morocco':220},{'Mauritania':221},{'Israel':222},{'Israel':223},{'Egypt':225},{'Egypt':226},{'Uganda':59},{'Tanzania':159},{'Kenya':157},{'Tanzania':193},{'Saudi Arabia':239},{'Yemen':240},{'Tanzania':242},{'Morocco':243},{'Mauritania':244},{'Sudan':245},{'South Sudan':246}]

filterCounties=[ "Botswana"]
cshapeArray=[]
for filteredCounty in filterCounties:
    
    for country in Countries:
        for country, cshape in country.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)

            if country == filteredCounty:
                cshapeArray.append(cshape)
                print(cshapeArray)


[155]


In [96]:

for country in cshapeArray:
    for date in arrayOfPrediction:
        print('data', date)
        print('country', country)
        print(df.loc[(date, country )]['wdi_sp_pop_totl'])

data 330
country 155
7862214.0
data 331
country 155
7862214.0
data 332
country 155
7862214.0
data 333
country 155
7862214.0
data 334
country 155
7862214.0
data 335
country 155
7862214.0
data 336
country 155
7862214.0
data 337
country 155
8126102.0
data 338
country 155
8126102.0
data 339
country 155
8126102.0
data 340
country 155
8126102.0
data 341
country 155
8126102.0
data 342
country 155
8126102.0
data 343
country 155
8126102.0
data 344
country 155
8126102.0
data 345
country 155
8126102.0
data 346
country 155
8126102.0
data 347
country 155
8126102.0
data 348
country 155
8126102.0
data 349
country 155
8397668.0
data 350
country 155
8397668.0
data 351
country 155
8397668.0
data 352
country 155
8397668.0
data 353
country 155
8397668.0
data 354
country 155
8397668.0
data 355
country 155
8397668.0
data 356
country 155
8397668.0
data 357
country 155
8397668.0
data 358
country 155
8397668.0
data 359
country 155
8397668.0
data 360
country 155
8397668.0
data 361
country 155
8675602.0
data 362

In [117]:
percentPopIncrease=0

for country in cshapeArray:
    for date in arrayOfPrediction:
        print('data', date)
        print('country', country)
        df.loc[(date, country ), 'wdi_sp_pop_totl']=(df.loc[(date, country )]['wdi_sp_pop_totl']*percentPopIncrease+df.loc[(date, country )]['wdi_sp_pop_totl'])
        print(df.loc[(date, country )]['wdi_sp_pop_totl'])

data 330
country 155
7862214.0
data 331
country 155
7862214.0
data 332
country 155
7862214.0
data 333
country 155
7862214.0
data 334
country 155
7862214.0
data 335
country 155
7862214.0
data 336
country 155
7862214.0
data 337
country 155
8126102.0
data 338
country 155
8126102.0
data 339
country 155
8126102.0
data 340
country 155
8126102.0
data 341
country 155
8126102.0
data 342
country 155
8126102.0
data 343
country 155
8126102.0
data 344
country 155
8126102.0
data 345
country 155
8126102.0
data 346
country 155
8126102.0
data 347
country 155
8126102.0
data 348
country 155
8126102.0
data 349
country 155
8397668.0
data 350
country 155
8397668.0
data 351
country 155
8397668.0
data 352
country 155
8397668.0
data 353
country 155
8397668.0
data 354
country 155
8397668.0
data 355
country 155
8397668.0
data 356
country 155
8397668.0
data 357
country 155
8397668.0
data 358
country 155
8397668.0
data 359
country 155
8397668.0
data 360
country 155
8397668.0
data 361
country 155
8675602.0
data 362

In [97]:
df2=df

In [104]:

for country in cshapeArray:
    for date in arrayOfPrediction:
        print('data', date)
        print('country', country)
        df2['wdi_sp_pop_totl'][df2.loc[(date,country)]['wdi_sp_pop_totl'] = 0
        print(df2.loc[(date, country )]['wdi_sp_pop_totl'])

SyntaxError: invalid syntax (<ipython-input-104-515bbdb60bfd>, line 5)

In [17]:
# model_from_pipeline_spec = all_cm_models_by_name["cm_sb_wdi_global"]

model_from_pipeline_spec1 = all_cm_models_by_name["cm_sb_wdi_global"]


models=[model_from_pipeline_spec1 ]
for model in models:
    model.periods = periods

In [18]:
cflong_acled_violence_ensemble = Ensemble(
    name="cflong_acled_violence_ensemble", 
    models=models, 
    outcome_type="prob", 
    col_outcome="greq_25_ged_best_sb", 
    method="average", 
    periods=periods
)
ensembles = [cflong_acled_violence_ensemble]


In [20]:
model_from_pipeline_spec1

{
  "name": "cm_sb_wdi_global",
  "col_outcome": "greq_25_ged_best_sb",
  "cols_features": [
    "in_africa",
    "wdi_ag_lnd_agri_zs",
    "wdi_ag_lnd_arbl_zs",
    "wdi_ag_lnd_frst_k2",
    "wdi_ag_lnd_prcp_mm",
    "wdi_ag_lnd_totl_k2",
    "wdi_ag_lnd_totl_ru_k2",
    "wdi_ag_prd_crop_xd",
    "wdi_ag_prd_food_xd",
    "wdi_ag_prd_lvsk_xd",
    "wdi_ag_srf_totl_k2",
    "wdi_ag_yld_crel_kg",
    "wdi_bg_gsr_nfsv_gd_zs",
    "wdi_bm_klt_dinv_wd_gd_zs",
    "wdi_bn_cab_xoka_gd_zs",
    "wdi_bx_gsr_ccis_zs",
    "wdi_bx_gsr_cmcp_zs",
    "wdi_bx_gsr_insf_zs",
    "wdi_bx_gsr_mrch_cd",
    "wdi_bx_gsr_tran_zs",
    "wdi_bx_gsr_trvl_zs",
    "wdi_bx_klt_dinv_cd_wd",
    "wdi_bx_klt_dinv_wd_gd_zs",
    "wdi_bx_trf_pwkr_dt_gd_zs",
    "wdi_dt_dod_dect_gn_zs",
    "wdi_dt_dod_pvlx_gn_zs",
    "wdi_dt_oda_oatl_kd",
    "wdi_dt_oda_odat_gn_zs",
    "wdi_dt_oda_odat_pc_zs",
    "wdi_dt_tds_dect_gn_zs",
    "wdi_eg_elc_accs_zs",
    "wdi_eg_use_elec_kh_pc",
    "wdi_eg_use_pcap_kg_oe",
    "wd

In [21]:
for model in models:
    model.periods = periods

In [22]:
for model in models:
    model.fit_estimators(df, populate_extras = False)

[2020-12-22 12:40:38,478] - views.apps.model.api:441 - INFO - Fitting estimators for cm_sb_wdi_global
[2020-12-22 12:40:38,479] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_wdi_global for period A step 1
[2020-12-22 12:40:38,605] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_wdi_global
[2020-12-22 12:40:38,605] - views.apps.model.api:422 - DEBUG - cm_sb_wdi_global downsampled away 0
[2020-12-22 12:40:38,606] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_wdi_global on 14850 rows
[2020-12-22 12:40:38,606] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_wdi_global
[2020-12-22 12:44:10,513] - views.apps.model.api:116 - DEBUG - Saving cm_sb_wdi_global A 1 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_wdi_global_A_1.joblib
[2020-12-22 12:44:19,759] - views.apps.model.api:119 - DEBUG - cm_sb_wdi_global saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_s

[2020-12-22 13:12:20,977] - views.apps.model.api:119 - DEBUG - cm_sb_wdi_global saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_wdi_global_A_36.joblib
[2020-12-22 13:12:20,988] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_wdi_global for period A step 38
[2020-12-22 13:12:21,087] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_wdi_global
[2020-12-22 13:12:21,088] - views.apps.model.api:422 - DEBUG - cm_sb_wdi_global downsampled away 0
[2020-12-22 13:12:21,088] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_wdi_global on 12852 rows
[2020-12-22 13:12:21,088] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_wdi_global
[2020-12-22 13:14:52,781] - views.apps.model.api:116 - DEBUG - Saving cm_sb_wdi_global A 38 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_wdi_global_A_38.joblib
[2020-12-22 13:15:01,224] - views.apps.model.api:119 - DEBUG - cm_sb_wdi_g

[2020-12-22 13:48:29,198] - views.apps.model.api:119 - DEBUG - cm_sb_wdi_global saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_wdi_global_B_30.joblib
[2020-12-22 13:48:29,214] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_wdi_global for period B step 36
[2020-12-22 13:48:29,336] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_wdi_global
[2020-12-22 13:48:29,337] - views.apps.model.api:422 - DEBUG - cm_sb_wdi_global downsampled away 0
[2020-12-22 13:48:29,337] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_wdi_global on 14904 rows
[2020-12-22 13:48:29,338] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_wdi_global
[2020-12-22 13:51:38,603] - views.apps.model.api:116 - DEBUG - Saving cm_sb_wdi_global B 36 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_wdi_global_B_36.joblib
[2020-12-22 13:51:47,853] - views.apps.model.api:119 - DEBUG - cm_sb_wdi_g

[2020-12-22 14:33:01,746] - views.apps.model.api:119 - DEBUG - cm_sb_wdi_global saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_wdi_global_C_24.joblib
[2020-12-22 14:33:01,755] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_wdi_global for period C step 30
[2020-12-22 14:33:01,882] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_wdi_global
[2020-12-22 14:33:01,882] - views.apps.model.api:422 - DEBUG - cm_sb_wdi_global downsampled away 0
[2020-12-22 14:33:01,883] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_wdi_global on 17820 rows
[2020-12-22 14:33:01,883] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_wdi_global
[2020-12-22 14:37:06,808] - views.apps.model.api:116 - DEBUG - Saving cm_sb_wdi_global C 30 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_wdi_global_C_30.joblib
[2020-12-22 14:37:19,272] - views.apps.model.api:119 - DEBUG - cm_sb_wdi_g

In [16]:
for model in models:
    
    # Uncalibrated predictions
    df_pred = model.predict(df)
    # assign_into_df takes care to only overwrite rows with actual values
    # This way we can keep all periods in the same df
    # It's also idempotent, no joining, so run as many times as you like. 
    df = assign_into_df(df_to=df, df_from=df_pred)
    
    df_pred = model.predict_calibrated(
        df=df.fillna(0), 
        period_calib=period_a,
        period_test=period_b,)
        
    df = assign_into_df(df_to=df, df_from=df_pred)
    df_pred = model.predict_calibrated(
        df=df.fillna(0), 
        period_calib=period_b,
        period_test=period_c,
    )
    df = assign_into_df(df_to=df, df_from=df_pred)
    
    # Calibrated predictions


[2020-11-19 10:21:32,224] - views.apps.model.api:552 - INFO - Predicting for cm_sb_cflong
[2020-11-19 10:21:32,226] - views.apps.model.api:553 - DEBUG - Predicting for cm_sb_cflong periods: [Period(name='A', train_start=121, train_end=396, predict_start=397, predict_end=432), Period(name='B', train_start=121, train_end=432, predict_start=433, predict_end=468), Period(name='C', train_start=121, train_end=480, predict_start=483, predict_end=520)]
[2020-11-19 10:21:32,267] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_A_1.joblib
[2020-11-19 10:21:38,654] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_A_3.joblib
[2020-11-19 10:21:44,810] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_A_6.joblib
[2020-11-19 10:21:51,378] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models

[2020-11-19 10:27:30,455] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_B_6.joblib
[2020-11-19 10:27:37,921] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_B_9.joblib
[2020-11-19 10:27:45,849] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_B_12.joblib
[2020-11-19 10:27:54,857] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_B_18.joblib
[2020-11-19 10:28:06,451] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_B_24.joblib
[2020-11-19 10:28:19,215] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_B_30.joblib
[2020-11-19 10:28:30,989] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong

[2020-11-19 10:32:53,020] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_cflong_18_calibrated
[2020-11-19 10:32:53,036] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_cflong_24_calibrated
[2020-11-19 10:32:53,049] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_cflong_30_calibrated
[2020-11-19 10:32:53,058] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_cflong_36_calibrated
[2020-11-19 10:32:53,069] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_cflong_38_calibrated
[2020-11-19 10:32:53,080] - views.utils.data:110 - DEBUG - Inserting col sc_cm_sb_cflong_calibrated
[2020-11-19 10:32:53,093] - views.apps.model.api:552 - INFO - Predicting for cm_sb_acled_violence
[2020-11-19 10:32:53,094] - views.apps.model.api:553 - DEBUG - Predicting for cm_sb_acled_violence periods: [Period(name='A', train_start=121, train_end=396, predict_start=397, predict_end=432), Period(name='B', train_start=121, train_end=432, predict_start=433, predict_end=468), Period

[2020-11-19 10:38:18,713] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_A_30.joblib
[2020-11-19 10:38:24,680] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_A_36.joblib
[2020-11-19 10:38:30,403] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_A_38.joblib
[2020-11-19 10:38:36,594] - views.apps.model.api:552 - INFO - Predicting for cm_sb_acled_violence
[2020-11-19 10:38:36,595] - views.apps.model.api:553 - DEBUG - Predicting for cm_sb_acled_violence periods: [Period(name='B', train_start=121, train_end=432, predict_start=433, predict_end=468)]
[2020-11-19 10:38:36,616] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_B_1.joblib
[2020-11-19 10:38:43,746] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/Op

[2020-11-19 10:42:14,563] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_acled_violence step 1
[2020-11-19 10:42:14,574] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_acled_violence step 3
[2020-11-19 10:42:14,587] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_acled_violence step 6
[2020-11-19 10:42:14,599] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_acled_violence step 9
[2020-11-19 10:42:14,611] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_acled_violence step 12
[2020-11-19 10:42:14,622] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_acled_violence step 18
[2020-11-19 10:42:14,644] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_acled_violence step 24
[2020-11-19 10:42:14,658] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_acled_violence step 30
[2020-11-19 10:42:14,667] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_acled_violence step 36
[2020-11-19 10:42:14,679] - views.apps.model.api:611 - DEBUG - Calib

In [17]:
for model in models:
    model.evaluate(df)

[2020-11-19 10:43:38,650] - views.apps.model.api:966 - INFO - Evaluating cm_sb_cflong
[2020-11-19 10:43:38,653] - views.apps.model.api:970 - DEBUG - Evaluating uncalibrated predictions for cm_sb_cflong period A step-combined
[2020-11-19 10:43:38,712] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_cflong period A step 1
[2020-11-19 10:43:38,755] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_cflong period A step 3
[2020-11-19 10:43:38,793] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_cflong period A step 6
[2020-11-19 10:43:38,830] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_cflong period A step 9
[2020-11-19 10:43:38,865] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_cflong period A step 12
[2020-11-19 10:43:38,899] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb

[2020-11-19 10:43:40,286] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_acled_violence period A step 12
[2020-11-19 10:43:40,320] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_acled_violence period A step 18
[2020-11-19 10:43:40,356] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_acled_violence period A step 24
[2020-11-19 10:43:40,395] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_acled_violence period A step 30
[2020-11-19 10:43:40,443] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_acled_violence period A step 36
[2020-11-19 10:43:40,492] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_acled_violence period A step 38
[2020-11-19 10:43:40,543] - views.apps.model.api:970 - DEBUG - Evaluating uncalibrated predictions for cm_sb_acled_violence period B step-combined
[

In [18]:
for model in models:
    print(model.name)
    #print(model.scores)
    print("EVAL SCORES:")
    print(json.dumps(model.scores, indent=2))
    print("FEATURE_IMPORTANCES")
    print(json.dumps(model.extras.feature_importances, indent=2))
    print("#"*80)
    

cm_sb_cflong
EVAL SCORES:
{
  "A": {
    "1": {
      "uncalibrated": {
        "average_precision": 0.7754431153761843,
        "area_under_roc": 0.9620479122471155,
        "brier": 0.04286612452860683
      },
      "calibrated": {}
    },
    "3": {
      "uncalibrated": {
        "average_precision": 0.7595176822459726,
        "area_under_roc": 0.9576590505814785,
        "brier": 0.043998182828007014
      },
      "calibrated": {}
    },
    "6": {
      "uncalibrated": {
        "average_precision": 0.74595661391252,
        "area_under_roc": 0.9523571108994122,
        "brier": 0.045168314853951244
      },
      "calibrated": {}
    },
    "9": {
      "uncalibrated": {
        "average_precision": 0.7346562588699618,
        "area_under_roc": 0.9472240906503855,
        "brier": 0.04661370021052189
      },
      "calibrated": {}
    },
    "12": {
      "uncalibrated": {
        "average_precision": 0.7268097416754156,
        "area_under_roc": 0.94051296722963,
        "b

In [18]:
type(df)
df.head(4)

acled_count_ns  acled_count_os  acled_count_pr  \
month_id country_id                                                   
1        40                     0.0             0.0             0.0   
         41                     0.0             0.0             0.0   
         42                     0.0             0.0             0.0   
         43                     0.0             0.0             0.0   

                     acled_count_sb  acled_dummy_ns  acled_dummy_os  \
month_id country_id                                                   
1        40                     0.0               0               0   
         41                     0.0               0               0   
         42                     0.0               0               0   
         43                     0.0               0               0   

                     acled_dummy_pr  acled_dummy_sb  cdum_1  cdum_10  ...  \
month_id country_id                                                   ...   
1        40                       0               0       0        0  ...   
         41                       0               0       0        0  ...   
         42                       0               0       0        0  ...   
         43                       0               0       0        0  ...   

                     ss_cm_sb_acled_violence_3_calibrated  \
month_id country_id                                         
1        40                                           NaN   
         41                                           NaN   
         42                                           NaN   
         43                                           NaN   

                     ss_cm_sb_acled_violence_6_calibrated  \
month_id country_id                                         
1        40                                           NaN   
         41                                           NaN   
         42                                           NaN   
         43                                           NaN   

                     ss_cm_sb_acled_violence_9_calibrated  \
month_id country_id                                         
1        40                                           NaN   
         41                                           NaN   
         42                                           NaN   
         43                                           NaN   

                     ss_cm_sb_acled_violence_12_calibrated  \
month_id country_id                                          
1        40                                            NaN   
         41                                            NaN   
         42                                            NaN   
         43                                            NaN   

                     ss_cm_sb_acled_violence_18_calibrated  \
month_id country_id                                          
1        40                                            NaN   
         41                                            NaN   
         42                                            NaN   
         43                                            NaN   

                     ss_cm_sb_acled_violence_24_calibrated  \
month_id country_id                                          
1        40                                            NaN   
         41                                            NaN   
         42                                            NaN   
         43                                            NaN   

                     ss_cm_sb_acled_violence_30_calibrated  \
month_id country_id                                          
1        40                                            NaN   
         41                                            NaN   
         42                                            NaN   
         43                                            NaN   

                     ss_cm_sb_acled_violence_36_calibrated  \
month_id country_id                              

In [25]:

for ensemble in ensembles:
    df_pred = ensemble.predict(
        df=df.fillna(0), 
        period_calib=period_b,
        period_test=period_c,
    )
    df = assign_into_df(df_to=df, df_from=df_pred)

[2020-11-19 11:26:48,011] - views.apps.model.api:814 - INFO - Predicting for cflong_acled_violence_ensemble
[2020-11-19 11:26:48,184] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_1
[2020-11-19 11:26:48,190] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_3
[2020-11-19 11:26:48,196] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_6
[2020-11-19 11:26:48,204] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_9
[2020-11-19 11:26:48,211] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_12
[2020-11-19 11:26:48,221] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_18
[2020-11-19 11:26:48,228] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_24
[2020-11-19 11:26:48,238] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_30
[2020-11-19 11:2

In [27]:
type(df)
df=df.fillna(0)
df.head(4)

acled_count_ns  acled_count_os  acled_count_pr  \
month_id country_id                                                   
1        40                     0.0             0.0             0.0   
         41                     0.0             0.0             0.0   
         42                     0.0             0.0             0.0   
         43                     0.0             0.0             0.0   

                     acled_count_sb  acled_dummy_ns  acled_dummy_os  \
month_id country_id                                                   
1        40                     0.0               0               0   
         41                     0.0               0               0   
         42                     0.0               0               0   
         43                     0.0               0               0   

                     acled_dummy_pr  acled_dummy_sb  cdum_1  cdum_10  ...  \
month_id country_id                                                   ...   
1        40                       0               0       0        0  ...   
         41                       0               0       0        0  ...   
         42                       0               0       0        0  ...   
         43                       0               0       0        0  ...   

                     ss_cflong_acled_violence_ensemble_3  \
month_id country_id                                        
1        40                                          0.0   
         41                                          0.0   
         42                                          0.0   
         43                                          0.0   

                     ss_cflong_acled_violence_ensemble_6  \
month_id country_id                                        
1        40                                          0.0   
         41                                          0.0   
         42                                          0.0   
         43                                          0.0   

                     ss_cflong_acled_violence_ensemble_9  \
month_id country_id                                        
1        40                                          0.0   
         41                                          0.0   
         42                                          0.0   
         43                                          0.0   

                     ss_cflong_acled_violence_ensemble_12  \
month_id country_id                                         
1        40                                           0.0   
         41                                           0.0   
         42                                           0.0   
         43                                           0.0   

                     ss_cflong_acled_violence_ensemble_18  \
month_id country_id                                         
1        40                                           0.0   
         41                                           0.0   
         42                                           0.0   
         43                                           0.0   

                     ss_cflong_acled_violence_ensemble_24  \
month_id country_id                                         
1        40                                           0.0   
         41                                           0.0   
         42                                           0.0   
         43                                           0.0   

                     ss_cflong_acled_violence_ensemble_30  \
month_id country_id                                         
1        40                                           0.0   
         41                                           0.0   
         42                                           0.0   
         43                                           0.0   

                     ss_cflong_acled_violence_ensemble_36  \
month_id country_id                                         
1        40                    

In [28]:
type(df)
# df.head(4)
# count = df["sc_cflong_acled_violence_ensemble"].isna().sum()
# count
# len(df)
# df.describe()
print(df[['sc_cflong_acled_violence_ensemble']].describe())

       sc_cflong_acled_violence_ensemble
count                       33048.000000
mean                            0.009693
std                             0.062317
min                             0.000000
25%                             0.000000
50%                             0.000000
75%                             0.000000
max                             0.807549


In [17]:
cflong_acled_violence_ensemble.col_sc

'sc_cflong_acled_violence_ensemble'

In [131]:
count = df["sc_cflong_acled_violence_ensemble"].isna().sum()
count

30996

In [76]:
# Tried to drop this columns but that didn't help
# df.drop(empty_cols,
#         axis=1,
#         inplace=True)

In [21]:
import pdb

In [29]:
# import pdb;  pdb.set_trace()
for ensemble in ensembles:
    ensemble.evaluate(df, period=periods_by_name["B"])
    ensemble.evaluate(df, period=periods_by_name["C"])

[2020-11-19 11:27:38,783] - views.apps.model.api:966 - INFO - Evaluating cflong_acled_violence_ensemble
[2020-11-19 11:27:38,786] - views.apps.model.api:970 - DEBUG - Evaluating uncalibrated predictions for cflong_acled_violence_ensemble period B step-combined
[2020-11-19 11:27:38,832] - views.apps.model.api:992 - DEBUG - Evaluating calibrated predictions for cflong_acled_violence_ensemble period B step-combined
[2020-11-19 11:27:38,835] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cflong_acled_violence_ensemble period B step 1
[2020-11-19 11:27:38,881] - views.apps.model.api:1033 - DEBUG - Evaluating calibrated predictions for cflong_acled_violence_ensemble period B step 1
[2020-11-19 11:27:38,887] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cflong_acled_violence_ensemble period B step 3
[2020-11-19 11:27:38,931] - views.apps.model.api:1033 - DEBUG - Evaluating calibrated predictions for cflong_acled_violence_ensemble 

In [30]:
for ensemble in ensembles:
    print(ensemble.name)
    print("Weights:")
    print(json.dumps(ensemble.weights, indent=2))
    print("Eval scores:")
    print(json.dumps(ensemble.evaluation.scores, indent=2))
    print("#"*80)

cflong_acled_violence_ensemble
Weights:
{
  "B": {
    "1": {
      "cm_sb_cflong": 0.5,
      "cm_sb_acled_violence": 0.5
    },
    "3": {
      "cm_sb_cflong": 0.5,
      "cm_sb_acled_violence": 0.5
    },
    "6": {
      "cm_sb_cflong": 0.5,
      "cm_sb_acled_violence": 0.5
    },
    "9": {
      "cm_sb_cflong": 0.5,
      "cm_sb_acled_violence": 0.5
    },
    "12": {
      "cm_sb_cflong": 0.5,
      "cm_sb_acled_violence": 0.5
    },
    "18": {
      "cm_sb_cflong": 0.5,
      "cm_sb_acled_violence": 0.5
    },
    "24": {
      "cm_sb_cflong": 0.5,
      "cm_sb_acled_violence": 0.5
    },
    "30": {
      "cm_sb_cflong": 0.5,
      "cm_sb_acled_violence": 0.5
    },
    "36": {
      "cm_sb_cflong": 0.5,
      "cm_sb_acled_violence": 0.5
    },
    "38": {
      "cm_sb_cflong": 0.5,
      "cm_sb_acled_violence": 0.5
    }
  }
}
Eval scores:
{
  "A": {
    "1": {
      "uncalibrated": {},
      "calibrated": {}
    },
    "3": {
      "uncalibrated": {},
      "calibrated": 

In [101]:
cflong_acled_violence_ensemble.col_sc_calibrated

'sc_cflong_acled_violence_ensemble'